## Notebook responsável pela geração do mapa utilizado pelo SVAA

In [ ]:
import os

import folium
import numpy as np
import pandas as pd
import requests as req
from resolve_path import ajuste_path

### Carrega o Dataset do mapa

In [ ]:
# se não existir o folder 'predicao', ele cria
path_dataset_para_mapa = "data/util/"

path_dataset_para_mapa = ajuste_path(path_dataset_para_mapa)
if not os.path.exists(path_dataset_para_mapa + 'predicao'):
    os.makedirs(path_dataset_para_mapa + 'predicao')

df_mapa = pd.read_csv(path_dataset_para_mapa + 'dataset_mapa.csv', sep=',')

### Cria datasets a partir do df_mapa para visualização de diferentes camadas no mapa

In [ ]:
df_mapa_score_0 = df_mapa.query("`Probabilidade` == 0")

df_mapa_score_entre_0_e_25 = df_mapa.query(
    "`Probabilidade` > 0 and `Probabilidade` < 0.25"
)

df_mapa_score_entre_25_e_50 = df_mapa.query(
    "`Probabilidade` >= 0.25 and `Probabilidade` < 0.5"
)

df_mapa_score_entre_50_e_75 = df_mapa.query(
    "`Probabilidade` >= 0.5 and `Probabilidade` < 0.75"
)

df_mapa_score_entre_75_e_100 = df_mapa.query(
    "`Probabilidade` >= 0.75 and `Probabilidade` <= 1"
)

### Definição das cores que serão utilizadas no mapa 

In [ ]:
corAmarela = "#FDBE85"
corBeje = "#FD8D3C"
corLaranja = "#E6550D"
corLaranjaEscuro = "#A63603"

cores_intermediarias_gradiente = [
    corAmarela, corBeje, corLaranja, corLaranjaEscuro]


def get_color_discrete(score):
    if score is np.nan:
        return "#000000"
    elif score < 0.25:
        return corAmarela
    elif score < 0.5:
        return corBeje
    elif score < 0.75:
        return corLaranja
    else:
        return corLaranjaEscuro

### Carrega atributos necessários para a exibição do mapa

In [ ]:
# calcula onde vai começar centralizado o mapa
media_x = df_mapa["Latitude"].mean()
media_y = df_mapa["Longitude"].mean()

# carrega os polígonos dos Estados do Brasil
url_poly_brasil = "https://servicodados.ibge.gov.br/api/v3/malhas/paises/BR?formato=application/vnd.geo+json&qualidade=maxima&intrarregiao=UF"

headers = {"Accept": "application/vnd.geo+json"}

mapa_brasil = req.get(url_poly_brasil, headers=headers)

poly_brasil = mapa_brasil.json()

### Funções utilizadas pelo mapa

In [ ]:
# Função que adiciona camadas ao mapa


def adiciona_camadas(map_object, data, nome_camada):
    camada = folium.FeatureGroup(name=nome_camada)
    for _, row in data.iterrows():
        color = get_color_discrete(row["Probabilidade"])
        popup_content = f"""
            <div>
                <div style="font-size: 1.1em;">
                    <b>Local:</b> {row['Local de instalação']}<br>
                </div>
                <div style="margin-top: 10px;">
                    <b>Quantidade de Acidentes em {(row['Mes'])}/{(row['Ano'])}:</b> {round(row['Quantidade de Acidentes'], 2)}<br>
                    <b>Quantidade de Acidentes de Alto Potencial em {(row['Mes'])}/{(row['Ano'])}:</b> {round(row['Quantidade de Acidentes de Alto Potencial'])}<br>
                    <b>HH total por mês em {(row['Mes'])}/{(row['Ano'])}:</b> {round(row['HH por mes'], 2)}<br>
                    <b>Probabilidade:</b> {round(row['Probabilidade'], 2)}
                </div>
            </div>
        """
        folium.Marker(
            location=(row["Latitude"], row["Longitude"]),
            icon=folium.DivIcon(
                html=f"""
                <div style="
                    background-color: {color};
                    color: white;
                    border-radius: 50%;
                    width: 50px;
                    height: 50px;
                    display: flex;
                    justify-content: center;
                    align-items: center;
                    font-size: 14px;
                    font-weight: bold;
                    transform: translate(-50%, -50%);
                ">
                    {round(round(row['Probabilidade'], 2)*100, 2)}%
                </div>
            """
            ),
            popup=folium.Popup(popup_content, max_width=2650),
            zIndexOffset=round(row["Probabilidade"] * 100),
        ).add_to(camada)

    map_object.add_child(camada)


# Função que define o período de tempo para a visualização do mapa
def filtra_df_por_tempo(df, mes, ano):
    df = df.query("`Mes` == @mes and `Ano` == @ano")
    return df

### Criação do Mapa

In [ ]:
# Criar o mapa centrado nas coordenadas especificadas
mapa = folium.Map(
    location=[media_x, media_y],  # Use 'location' em vez de 'center'
    zoom_start=5,
    control_scale=True,
    dragging=True,
)


# Adiciona html de legenda ao mapa
caminho_legenda_html = ajuste_path("assets/legenda_mapa.html")
with open(caminho_legenda_html, "r", encoding="utf-8") as arquivo:
    legenda_html = arquivo.read()

legenda_css = """
<div style="
    position: fixed;
    bottom: 20px;
    right: 10px;
    z-index: 1000;
    background-color: rgba(255, 255, 255, 0.8);
    padding: 10px;
    border-radius: 5px;
    box-shadow: 2px 2px 5px rgba(0, 0, 0, 0.4);
">
    {conteudo}
</div>
""".format(conteudo=legenda_html)

# Adiciona o HTML com estilo ao mapa
mapa.get_root().html.add_child(folium.Element(legenda_css))

# Adicionar a camada de polígonos de estados do Brasil
camada_poligonos_brasil = folium.FeatureGroup(name="Polígonos do Brasil")
folium.GeoJson(
    poly_brasil,
    zoom_on_click=True,
    style_function=lambda feature: {
        "fillColor": "#c8d977",
        "color": "black",
        "weight": 2,
        "dashArray": "5, 5",
        "fillOpacity": 0.15,
    },
).add_to(camada_poligonos_brasil)
mapa.add_child(camada_poligonos_brasil)

# Adicionar camadas de visualização ao mapa
adiciona_camadas(mapa, filtra_df_por_tempo(df_mapa, 6, 2024,),
                 "Exibir probabilidade de 06/2024")
adiciona_camadas(mapa, filtra_df_por_tempo(df_mapa, 6, 2024).query(
    "`Quantidade de Acidentes` >= 1"), "Exibir subestações com acidentes")

folium.LayerControl().add_to(mapa)

### Salva o mapa dentro do folder 'mapas_html'

In [ ]:
# Gerar o HTML do mapa como uma string
mapa_html = mapa._repr_html_()

# Salvar a string em um arquivo com encoding UTF-8
caminho_mapa = "mapas_html/mapa_mvp.html"
if not os.path.exists("mapas_html"):
    os.makedirs("mapas_html")
with open(caminho_mapa, "w", encoding="utf-8") as arquivo:
    arquivo.write(mapa_html)

# Exibe o mapa
mapa